In [1]:
import pandas as pd
import numpy as np
import time as TIME
import datetime as dt
from datetime import date
from datetime import timedelta
import math
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_excel('fno_stocks.xlsx', header= 1)
Tickers = (df["Symbol"])
Tickers = Tickers.astype(str)

Tickers = Tickers.drop(Tickers[Tickers.values == "BANKNIFTY"].index.values[0], axis =0)
Tickers = Tickers.drop(Tickers[Tickers.values == "NIFTY"].index.values[0], axis =0)
Tickers = Tickers.reset_index(drop = True)
Tickers

0            IDEA
1      COROMANDEL
2            ZEEL
3       NESTLEIND
4        ASHOKLEY
          ...    
167    APOLLOHOSP
168        INDIGO
169    CANFINHOME
170    ICICIPRULI
171       SBILIFE
Name: Symbol, Length: 172, dtype: object

In [3]:
total_data = pd.read_csv("Day_Wise_Stats_Combined.csv", index_col = 0, parse_dates = [2])
total_data

,AD_Ratio,Date,Stock1,Stock2,Stock3,Stock4,Stock5
0,0.951220,2017-01-02 00:00:00+05:30,BHARATFORG,ASTRAL,MOTHERSUMI,PETRONET,BHEL
1,1.461538,2017-01-03 00:00:00+05:30,IOC,SUNTV,DEEPAKNTR,PNB,ESCORTS
2,1.222222,2017-01-04 00:00:00+05:30,NMDC,COFORGE,IPCALAB,INDUSTOWER,M&MFIN
3,0.975309,2017-01-05 00:00:00+05:30,JSWSTEEL,COROMANDEL,IOC,ADANIPORTS,WIPRO
4,0.860465,2017-01-06 00:00:00+05:30,PFC,APOLLOTYRE,DLF,CANBK,CANFINHOME
...,...,...,...,...,...,...,...
1202,0.722772,2021-09-17 09:15:00+05:30,POLYCAB,INDIGO,JUBLFOOD,BAJAJFINSV,MCDOWELL-N
1203,5.692308,2021-09-20 09:15:00+05:30,INDHOTEL,GMRINFRA,PVR,ZEEL,ASHOKLEY
1204,0.251799,2021-09-21 09:15:00+05:30,ONGC,GODREJPROP,GRASIM,JSWSTEEL,PVR
1205,1.597015,2021-09-22 09:15:00+05:30,ZEEL,INDHOTEL,BALKRISIND,APOLLOTYRE,TECHM


In [12]:
%%time
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
Daily_pct_change = pd.DataFrame()
current_date = date(2017,1,1)
todays_date  = date(2020,8,1)
current_date = pd.Timestamp(current_date, tz =19800)
flag = 0
while(current_date.date() < todays_date):
    start = dt.datetime.now()
    pct_change = {}
    
    Daily_pct_change = pd.read_csv("Short_Selling_4.csv", index_col = 0,  parse_dates = [2])
    
    if(flag==0):
        current_date = Daily_pct_change.Date.iloc[-1] + timedelta(days =1)
        flag = 1
        if(weekDays[current_date.weekday()] == "Saturday" or weekDays[current_date.weekday()] == "Sunday"):
            print(str(current_date) + " : Started")
            print(str(current_date) + " : Finished") 
            current_date = current_date + timedelta(days =1)
            if(weekDays[current_date.weekday()] == "Saturday" or weekDays[current_date.weekday()] == "Sunday"):
                print(str(current_date) + " : Started")
                print(str(current_date) + " : Finished") 
                current_date = current_date + timedelta(days =1)    
                
    if(total_data[total_data.Date == current_date].AD_Ratio.iloc[-1] > 0.8):
        print(str(current_date) + " : AD Greater than 0.8 ")
        index = total_data[total_data.Date == current_date].index[0]
        current_date = total_data.Date.iloc[index + 1]
        continue
        
    print(str(current_date) + " : Started")
    for ticker in Tickers:
        daily_data = pd.read_csv("Historical_Data/" + ticker + ".csv", index_col = 0, parse_dates = [1])
        daily_data = daily_data[daily_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
        daily_data = daily_data[daily_data.date >= pd.Timestamp(current_date)]
        daily_data = daily_data.reset_index(drop = True)
        if(daily_data.empty == True):
            continue
        change_value = (daily_data[daily_data["date"] == daily_data["date"].iloc[0] + timedelta(minutes = 60)].close.values[0] - daily_data.close[0])/daily_data.close[0] 
        pct_change[ticker] = change_value    
    
    top_gainers = dict(sorted(pct_change.items(), key=lambda x: x[1], reverse=True))
    change = pd.DataFrame()
    change["Tickers"] = top_gainers.keys()
    change["PCT_Change"] = top_gainers.values()
    
    if(change.empty == True):
        print(str(current_date) + " : Finished") 
        continue
    gain_number = change[change["PCT_Change"] > 0].shape[0] + 1 
    loss_number = change[change["PCT_Change"] <= 0].shape[0] + 1
    ad_ratio    = gain_number/loss_number
    end = dt.datetime.now() - start
    print("Time Required to calculate pct change: " + str(end)) 
    print("PCT Change Calculated!")
    for i in range(5):
        print(change["Tickers"].iloc[-(i+1)])

    new_row = {"Date": current_date, "Stock1": change["Tickers"].iloc[-1], "Stock2" : change["Tickers"].iloc[-2], "Stock3": change["Tickers"].iloc[-3], "Stock4": change["Tickers"].iloc[-4],"Stock5": change["Tickers"].iloc[-5] , "AD_Ratio":ad_ratio }
    Daily_pct_change = Daily_pct_change.append(new_row, ignore_index = True )
    Daily_pct_change.to_csv("Short_Selling_4.csv")
    
    print(str(current_date) + " : Finished")
    index = total_data[total_data.Date == current_date].index[0]
    current_date = total_data.Date.iloc[index + 1]

2020-05-05 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:42.069315
PCT Change Calculated!
CHOLAFIN
BHEL
ICICIGI
APOLLOTYRE
PIDILITIND
2020-05-05 09:15:00+05:30 : Finished
2020-05-06 09:15:00+05:30 : AD Greater than 0.8 
2020-05-07 09:15:00+05:30 : AD Greater than 0.8 
2020-05-08 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:40.145761
PCT Change Calculated!
TORNTPOWER
BHARATFORG
NTPC
APOLLOTYRE
M&M
2020-05-08 09:15:00+05:30 : Finished
2020-05-11 09:15:00+05:30 : AD Greater than 0.8 
2020-05-12 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:38.586701
PCT Change Calculated!
GAIL
DEEPAKNTR
JINDALSTEL
STAR
SRTRANSFIN
2020-05-12 09:15:00+05:30 : Finished
2020-05-13 09:15:00+05:30 : Started
Time Required to calculate pct change: 0:03:38.706247
PCT Change Calculated!
VEDL
VOLTAS
MARUTI
INDIAMART
NTPC
2020-05-13 09:15:00+05:30 : Finished
2020-05-14 09:15:00+05:30 : AD Greater than 0.8 
2020-05-15 09:15:00+05:30 : AD Greater tha

In [8]:
current_date = date(2020,5,4)
current_date = pd.Timestamp(current_date, tz =19800)
Daily_pct_change = pd.DataFrame()
#todays_date = dt.datetime.now().date()
todays_date  = date(2021,9,23)
pct_change = {}
start = dt.datetime.now()
print(str(current_date) + " : Started")
for ticker in Tickers:
    daily_data = pd.read_csv("Historical_Data/" + ticker + ".csv", index_col = 0, parse_dates = [1])
    daily_data = daily_data[daily_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
    daily_data = daily_data[daily_data.date >= pd.Timestamp(current_date)]
    daily_data = daily_data.reset_index(drop = True)
    if(daily_data.empty == True):
        continue
    change_value = (daily_data[daily_data["date"] == daily_data["date"].iloc[0] + timedelta(minutes = 60)].close.values[0] - daily_data.close[0])/daily_data.close[0] 
    pct_change[ticker] = change_value
    #print(change_value)

top_gainers = dict(sorted(pct_change.items(), key=lambda x: x[1], reverse=True))
change = pd.DataFrame()
change["Tickers"] = top_gainers.keys()
change["PCT_Change"] = top_gainers.values()
gain_number = change[change["PCT_Change"] > 0].shape[0] + 1 
loss_number = change[change["PCT_Change"] <= 0].shape[0] + 1
ad_ratio    = gain_number/loss_number
end = dt.datetime.now() - start
print("Time Required to calculate pct change: " + str(end)) 
print("PCT Change Calculated!")
for i in range(5):
    print(change["Tickers"].iloc[-(i+1)])
    
new_row = {"Date": current_date, "Stock1": change["Tickers"].iloc[-1], "Stock2" : change["Tickers"].iloc[-2], "Stock3": change["Tickers"].iloc[-3], "Stock4": change["Tickers"].iloc[-4],"Stock5": change["Tickers"].iloc[-5] , "AD_Ratio":ad_ratio }
Daily_pct_change = Daily_pct_change.append(new_row, ignore_index = True )
Daily_pct_change.to_csv("Short_Selling_4.csv")

date_data = pd.read_csv("Historical_Data/" + change["Tickers"][0] + ".csv", index_col = 0, parse_dates = [1])
date_data = date_data[date_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
date_data = date_data[date_data.date >= pd.Timestamp(current_date)]
date_data = date_data.reset_index(drop = True)
print(str(current_date) + " : Finished")
current_date = date_data.date.iloc[-1]


2020-05-04 00:00:00+05:30 : Started
Time Required to calculate pct change: 0:03:39.568457
PCT Change Calculated!
MANAPPURAM
MUTHOOTFIN
ALKEM
L&TFH
M&MFIN
2020-05-04 00:00:00+05:30 : Finished


In [6]:
for i in range(5):
    print(change["Tickers"].iloc[-(i+1)])
    
new_row = {"Date": current_date, "Stock1": change["Tickers"].iloc[-1], "Stock2" : change["Tickers"].iloc[-2], "Stock3": change["Tickers"].iloc[-3], "Stock4": change["Tickers"].iloc[-4],"Stock5": change["Tickers"].iloc[-5] , "AD_Ratio":ad_ratio }
Daily_pct_change = Daily_pct_change.append(new_row, ignore_index = True )
Daily_pct_change.to_csv("Short_Selling.csv")

date_data = pd.read_csv("Historical_Data/" + change["Tickers"][0] + ".csv", index_col = 0, parse_dates = [1])
date_data = date_data[date_data.date <= pd.Timestamp(current_date) + timedelta(days =1, minutes = 555)]
date_data = date_data[date_data.date >= pd.Timestamp(current_date)]
date_data = date_data.reset_index(drop = True)
current_date = date_data.date.iloc[-1]
print(str(current_date) + " : Finished")


IndexError: single positional indexer is out-of-bounds